In [ ]:
import sympy as sp
from sympy.abc import s

from pytc2.sistemas_lineales import analyze_sys, parametrize_sos, pretty_print_lti, pretty_print_bicuad_omegayq, tf2sos_analog, pretty_print_SOS
from pytc2.general import print_latex, print_subtitle

In [ ]:
Q_BP = 5

